In [1]:
from PIL import Image
import pandas as pd
import numpy as np
import cv2 as cv
from glob import glob
from sklearn.model_selection import train_test_split
from mitosplit_net import preprocessing, augmentation, evaluation, training, plotting, util
from albumentations import Compose, Rotate, RandomRotate90, HorizontalFlip, Flip, ElasticTransform, GaussNoise, RandomCrop, Resize
from tqdm import tqdm
import os, sys
from os import path
import random as r
from import_augmentation_function import import_fun, aug_fun, import_aug_fun
import matplotlib.pyplot as plt
import tensorflow as tf 

In [2]:
## IMPORTS ALL IMAGES WITHIN THE FOLDER "imager_dir" ##

files_dir = r'C:\Users\roumba\Documents\Software\deep-events\training_data'
images_dir = '220915_mtstaygold_cos7_ZEISS_bf_pos'
joined_path = os.path.join(files_dir, images_dir)
img_size = 256
data_ratio=0.1
data_split_state = None 
number_of_augmentations = 5 

#for some reason, even though it should have been deleted, the first two arrays of axis0 are zeros?# 


In [14]:
## THIS WORKS FOR A NETWORK WITH NO MEMORY ##

all_image_array, all_image_array_gauss = import_fun(joined_path, files_dir, images_dir)
augmentation_data, data_val, augmentation_data_gauss, data_gauss_val =  train_test_split(all_image_array, all_image_array_gauss, 
                                                                                                       test_size=data_ratio, random_state=data_split_state)
data_aug = augmentation_data
data_gauss_aug = augmentation_data_gauss
for j in range(number_of_augmentations):
    augment_data, augment_data_gauss = aug_fun(augmentation_data, augmentation_data_gauss)
    data_aug = np.concatenate((data_aug, augment_data))
    data_gauss_aug = np.concatenate((data_gauss_aug,augment_data_gauss))


100%|██████████| 519/519 [00:00<00:00, 1457.18it/s]


In [10]:
np.shape(data_aug)

(3114, 256, 256)

In [ ]:
## THIS WORKS FOR MEMORY CONSERVATION ##

all_data_aug, all_data_gauss_aug, all_image_array, all_image_array_gauss = import_aug_fun(joined_path, files_dir, images_dir)
np.shape(all_data_aug)

## DATA SPLIT IN VALIDATION AND AUGMENTATION ##

val_size=int(data_ratio * np.size(all_data_aug,0))
rand_arr= np.zeros(val_size)
aug_size= np.size(all_data_aug,0) - val_size
data_val= np.zeros((val_size, 256,256))
data_gauss_val= np.zeros((val_size, 256,256))
x = list(range(np.size(all_data_aug,0)-val_size))

random_num_row = np.random.choice(x,val_size, replace=False)

for i in range(val_size):
    x= random_num_row[i]
    data_val[i,:,:]= all_image_array[x,:,:]
    data_gauss_val[i,:,:]= all_image_array_gauss[x,:,:]
    if i==0:
        data_aug = np.delete(all_data_aug, x, axis=0)
        data_gauss_aug = np.delete(all_data_gauss_aug, x, axis=0)
    else:
        data_aug = np.delete(data_aug, x, axis=0)
        data_gauss_aug = np.delete(data_gauss_aug, x, axis=0)

100%|██████████| 2/2 [00:00<?, ?it/s]


(578, 256, 256)

In [4]:
## NORMALIZATION ##    

for framenumber in range(np.size(data_val, 0)):
    # k is the bit of data that we want to set to background and this should be reconsidered since maybe just quoting a number isn't very productive#
    k=0.1 
    kk=1/(1-k)

    # validation data #
    data_val = data_val/np.max(data_val)                                               
    data_val = data_val-k
    data_val[data_val < 0] = 0   
    data_val = data_val*kk

    data_gauss_val = data_gauss_val/np.max(data_gauss_val) 
    data_gauss_val = data_gauss_val-k
    data_gauss_val[data_gauss_val < 0] = 0     
    data_gauss_val = data_gauss_val*kk                                             
    

for framenumber in range(np.size(data_aug,0)):

    # augmentation data #
    data_aug = data_aug/np.max(data_aug)                                               
    data_aug = data_aug-k
    data_aug[data_aug < 0] = 0   
    data_aug = data_aug*kk

    data_gauss_aug = data_gauss_aug/np.max(data_gauss_aug) 
    data_gauss_aug = data_gauss_aug-k
    data_gauss_aug[data_gauss_aug < 0] = 0     
    data_gauss_aug = data_gauss_aug*kk     

In [16]:
data_aug = data_aug/np.max(data_aug) 


np.max(data_aug)                                                       

1.0

In [6]:
## SHUFFLE ##

data_val = r.shuffle(data_val)
data_gauss_val = r.shuffle(data_gauss_val)
data_aug = r.shuffle(data_aug)
data_gauss_aug = r.shuffle(data_gauss_aug)

In [13]:
#Define GPU device where the code will run on

gpu = tf.config.list_physical_devices('GPU')[0]
print(gpu)
tf.config.experimental.set_memory_growth(gpu, True)
gpu = tf.device('GPU:0/') 

base_dir = r'C:\Users\roumba\Documents\Software\deep-events'
model_path = base_dir+'\Models'
print('model_path:', model_path)

PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
model_path: C:\Users\roumba\Documents\Software\deep-events\Models


In [17]:
with gpu:
  nb_filters = 8
  firstConvSize = 9
  batch_size = [8, 16, 32, 256]
  model, history= {}, {}
  
  b=batch_size[1]
  model_name = 'ref_f%i_c%i_b%i'%(nb_filters, firstConvSize, b)
  print('Model:', model_name)
  model[model_name] = training.create_model(nb_filters, firstConvSize)
  history[model_name] = training.train_model(model[model_name], data_aug, data_gauss_aug, batch_size=b)

Model: ref_f8_c9_b16
* Start Encoder Section *
* Start Center Section *
* Start Decoder Section *
Epoch 1/20
21/21 - 9s - loss: 0.9240 - binary_accuracy: 0.3355 - val_loss: 0.7136 - val_binary_accuracy: 0.2338 - 9s/epoch - 406ms/step
Epoch 2/20
21/21 - 6s - loss: 0.7911 - binary_accuracy: 0.3902 - val_loss: 0.7290 - val_binary_accuracy: 0.0677 - 6s/epoch - 274ms/step
Epoch 3/20
21/21 - 6s - loss: 0.6038 - binary_accuracy: 0.7447 - val_loss: 0.6527 - val_binary_accuracy: 0.9849 - 6s/epoch - 276ms/step
Epoch 4/20
21/21 - 6s - loss: 0.5030 - binary_accuracy: 0.9526 - val_loss: 0.6169 - val_binary_accuracy: 0.9960 - 6s/epoch - 274ms/step
Epoch 5/20
21/21 - 6s - loss: 0.4563 - binary_accuracy: 0.9703 - val_loss: 0.5848 - val_binary_accuracy: 0.9960 - 6s/epoch - 274ms/step
Epoch 6/20
21/21 - 6s - loss: 0.4261 - binary_accuracy: 0.9827 - val_loss: 0.5569 - val_binary_accuracy: 0.9955 - 6s/epoch - 272ms/step
Epoch 7/20
21/21 - 6s - loss: 0.3990 - binary_accuracy: 0.9894 - val_loss: 0.5338 - va

In [18]:
folder_name = list(model.keys())

util.save_model(model, model_path, ['model']*len(model), folder_name)
util.save_pkl(history, model_path, ['history']*len(model), folder_name)


Saving C:\Users\roumba\Documents\Software\deep-events\Modelsref_f8_c9_b16/model.h5
Done.

Saving C:\Users\roumba\Documents\Software\deep-events\Modelsref_f8_c9_b16/history
Done.

Saving C:\Users\roumba\Documents\Software\deep-events\Modelsref_f8_c9_b16/frames_test
Done.
